In [12]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.preprocessing import image

# Set dataset paths
base_dir = r"C:\Users\Sagar Panchal\OneDrive\Desktop\Cows"  

# Image properties
IMG_HEIGHT = 150
IMG_WIDTH = 150
BATCH_SIZE = 32

# Data preprocessing with augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 80% train, 20% validation
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Number of cow classes
num_classes = len(train_generator.class_indices)
print("Class labels:", train_generator.class_indices)

# Model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Output: one node per cow
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator
)

# Save the model
model.save("cow_id.h5")
print("✅ Model saved as cow_identification_model.h5")

# Reload the saved model for prediction
model = tf.keras.models.load_model("cow_id.h5")

# Get class labels in correct order
class_indices = train_generator.class_indices
# Reverse mapping from index to label
class_labels = [None] * len(class_indices)
for label, index in class_indices.items():
    class_labels[index] = label

# Prediction function
def predict_cow_id(img_path):
    # Load and preprocess image
    img = image.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Make prediction
    prediction = model.predict(img_array)
    predicted_index = np.argmax(prediction)
    predicted_label = class_labels[predicted_index]
    confidence = np.max(prediction) * 100

    print(f"📷 Image: {img_path}")
    print(f"🐄 Predicted Cow ID: {predicted_label}")
    print(f"🎯 Confidence: {confidence:.2f}%")

# === Test with your image ===
test_image_path = r"C:\Users\Sagar Panchal\OneDrive\Desktop\Cows\COW (11)\WhatsApp Image 2025-06-05 at 17.36.52_457502cd.jpg"
predict_cow_id(test_image_path)


Found 171 images belonging to 33 classes.
Found 30 images belonging to 33 classes.
Class labels: {'COW': 0, 'COW (10)': 1, 'COW (11)': 2, 'COW (12)': 3, 'COW (13)': 4, 'COW (14)': 5, 'COW (15)': 6, 'COW (16)': 7, 'COW (17)': 8, 'COW (18)': 9, 'COW (19)': 10, 'COW (2)': 11, 'COW (20)': 12, 'COW (21)': 13, 'COW (22)': 14, 'COW (23)': 15, 'COW (24)': 16, 'COW (25)': 17, 'COW (26)': 18, 'COW (27)': 19, 'COW (28)': 20, 'COW (29)': 21, 'COW (3)': 22, 'COW (30)': 23, 'COW (31)': 24, 'COW (32)': 25, 'COW (33)': 26, 'COW (4)': 27, 'COW (5)': 28, 'COW (6)': 29, 'COW (7)': 30, 'COW (8)': 31, 'COW (9)': 32}


C:\Users\Sagar Panchal\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Sagar Panchal\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 634ms/step - accuracy: 0.0506 - loss: 3.7526 - val_accuracy: 0.0667 - val_loss: 3.4771
Epoch 2/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 504ms/step - accuracy: 0.0398 - loss: 3.4898 - val_accuracy: 0.0667 - val_loss: 3.4473
Epoch 3/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 548ms/step - accuracy: 0.0822 - loss: 3.4937 - val_accuracy: 0.0667 - val_loss: 3.4554
Epoch 4/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 522ms/step - accuracy: 0.0394 - loss: 3.4666 - val_accuracy: 0.0667 - val_loss: 3.3876
Epoch 5/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 507ms/step - accuracy: 0.1051 - loss: 3.3359 - val_accuracy: 0.1333 - val_loss: 3.3521
Epoch 6/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 504ms/step - accuracy: 0.0655 - loss: 3.3107 - val_accuracy: 0.1333 - val_loss: 3.2459
Epoch 7/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 513ms/step - accuracy: 0.0826 - loss: 3.2503 - val_accuracy: 0.1333 - val_loss: 3.2162
Epoch 8/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 510ms/step - accuracy: 0.0909 - loss: 3.1429 - val_accuracy: 0.1667 - val_loss:

✅ Model saved as cow_identification_model.h5


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
📷 Image: C:\Users\Sagar Panchal\OneDrive\Desktop\Cows\COW (11)\WhatsApp Image 2025-06-05 at 17.36.52_457502cd.jpg
🐄 Predicted Cow ID: COW (7)
🎯 Confidence: 53.20%


In [13]:
import tensorflow as tf

# Load the Keras model
model = tf.keras.models.load_model("cow_id.h5")

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# (Optional) Enable optimizations for smaller model size (recommended for mobile)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert the model
tflite_model = converter.convert()

# Save the .tflite model
with open("cow_id.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ TFLite model saved as cow_eye.tflite")

INFO:tensorflow:Assets written to: C:\Users\SAGARP~1\AppData\Local\Temp\tmpc9ddqibx\assets


INFO:tensorflow:Assets written to: C:\Users\SAGARP~1\AppData\Local\Temp\tmpc9ddqibx\assets


Saved artifact at 'C:\Users\SAGARP~1\AppData\Local\Temp\tmpc9ddqibx'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name='input_layer_10')
Output Type:
  TensorSpec(shape=(None, 33), dtype=tf.float32, name=None)
Captures:
  1837774663952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1837770851536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1837778834768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1837778834384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1837778834576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1837778832080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1837778829584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1837778832464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1837778831888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1837778835152: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅

In [7]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import json # Import json to save class indices

# Set dataset paths
# IMPORTANT: Adjust this path to where your cow images are located
base_dir = r"C:\Users\Sagar Panchal\OneDrive\Desktop\Cows"

# Image properties
IMG_HEIGHT = 150
IMG_WIDTH = 150
BATCH_SIZE = 32
EPOCHS = 15 # Define epochs as a variable

# Data preprocessing with augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 80% train, 20% validation
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest' # Added fill_mode for augmented images
)

print(f"Loading images from: {base_dir}")
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Number of cow classes
num_classes = len(train_generator.class_indices)
print("Class labels (index mapping):", train_generator.class_indices)

# --- SAVE CLASS INDICES ---
# This is crucial for your web app to correctly map predictions back to cow names
model_dir = "models" # Define a directory for models and related files
os.makedirs(model_dir, exist_ok=True)
with open(os.path.join(model_dir, "cow_class_indices.json"), "w") as f:
    json.dump(train_generator.class_indices, f)
print(f"✅ Class indices saved to {os.path.join(model_dir, 'cow_class_indices.json')}")

# Model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Output: one node per cow
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary() # Print model summary

# Train the model
print(f"\nStarting model training for {EPOCHS} epochs...")
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator
)

# Save the model
model.save(os.path.join(model_dir, "cow_id.h5"))
print(f"✅ Model saved as {os.path.join(model_dir, 'cow_id.h5')}")

print("\nTraining complete. You can now use 'cow_id.h5' and 'cow_class_indices.json' in your web app.")

# --- Optional: Test the reloaded model for a single prediction ---
# You can uncomment and modify this section if you want to test prediction
# with a single image directly from this script after training.

print("\n--- Testing reloaded model ---")
# Reload the saved model for prediction
model_reloaded = tf.keras.models.load_model(os.path.join(model_dir, "cow_id.h5"))

# Get class labels in correct order
# You need to load them back just like in the app.py
loaded_class_indices = {}
with open(os.path.join(model_dir, "cow_class_indices.json"), "r") as f:
    loaded_class_indices = json.load(f)

class_labels_reloaded = [None] * len(loaded_class_indices)
for label, index in loaded_class_indices.items():
    class_labels_reloaded[index] = label

# Prediction function for testing
def predict_cow_id_test(img_path_test):
    try:
        # Load and preprocess image
        img = tf.keras.preprocessing.image.load_img(img_path_test, target_size=(IMG_HEIGHT, IMG_WIDTH))
        img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0) # Add batch dimension

        # Make prediction
        prediction = model_reloaded.predict(img_array)
        predicted_index = np.argmax(prediction[0]) # Get index from the first (and only) sample in the batch
        predicted_label = class_labels_reloaded[predicted_index]
        confidence = np.max(prediction[0]) * 100

        print(f"📷 Test Image: {img_path_test}")
        print(f"🐄 Predicted Cow ID: {predicted_label}")
        print(f"🎯 Confidence: {confidence:.2f}%")
    except Exception as e:
        print(f"Error during test prediction for {img_path_test}: {e}")

# === Test with your image ===
# IMPORTANT: Change this path to a real image in your dataset for testing
test_image_path = r"C:\Users\Sagar Panchal\OneDrive\Desktop\Cows\COW (11)\WhatsApp Image 2025-06-05 at 17.36.52_457502cd.jpg"
predict_cow_id_test(test_image_path)


Loading images from: C:\Users\Sagar Panchal\OneDrive\Desktop\Cows
Found 171 images belonging to 33 classes.
Found 30 images belonging to 33 classes.
Class labels (index mapping): {'COW': 0, 'COW (10)': 1, 'COW (11)': 2, 'COW (12)': 3, 'COW (13)': 4, 'COW (14)': 5, 'COW (15)': 6, 'COW (16)': 7, 'COW (17)': 8, 'COW (18)': 9, 'COW (19)': 10, 'COW (2)': 11, 'COW (20)': 12, 'COW (21)': 13, 'COW (22)': 14, 'COW (23)': 15, 'COW (24)': 16, 'COW (25)': 17, 'COW (26)': 18, 'COW (27)': 19, 'COW (28)': 20, 'COW (29)': 21, 'COW (3)': 22, 'COW (30)': 23, 'COW (31)': 24, 'COW (32)': 25, 'COW (33)': 26, 'COW (4)': 27, 'COW (5)': 28, 'COW (6)': 29, 'COW (7)': 30, 'COW (8)': 31, 'COW (9)': 32}
✅ Class indices saved to models\cow_class_indices.json


C:\Users\Sagar Panchal\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       4,735,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 33)                  │           4,257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,832,609 (18.43 MB)

...

Epoch 1/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 932ms/step - accuracy: 0.0107 - loss: 3.7105

C:\Users\Sagar Panchal\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.0116 - loss: 3.7020 - val_accuracy: 0.1000 - val_loss: 3.4696
Epoch 2/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 883ms/step - accuracy: 0.0305 - loss: 3.4726 - val_accuracy: 0.0667 - val_loss: 3.3949
Epoch 3/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 885ms/step - accuracy: 0.0688 - loss: 3.4164 - val_accuracy: 0.1000 - val_loss: 3.2810
Epoch 4/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 915ms/step - accuracy: 0.0978 - loss: 3.2831 - val_accuracy: 0.1000 - val_loss: 3.2046
Epoch 5/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 896ms/step - accuracy: 0.1423 - loss: 3.2452 - val_accuracy: 0.1000 - val_loss: 3.0133
Epoch 6/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 860ms/step - accuracy: 0.1700 - loss: 3.0997 - val_accuracy: 0.2000 - val_loss: 2.8677
Epoch 7/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 863ms/step - accuracy: 0.1644 - loss: 2.8933 - val_accuracy: 0.2667 - val_loss: 2.6929
Epoch 8/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 875ms/step - accuracy: 0.1791 - loss: 2.8385 - val_accuracy: 0.4333 - val_loss: 2.4345
Epoch

✅ Model saved as models\cow_id.h5

Training complete. You can now use 'cow_id.h5' and 'cow_class_indices.json' in your web app.

--- Testing reloaded model ---


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step
📷 Test Image: C:\Users\Sagar Panchal\OneDrive\Desktop\Cows\COW (11)\WhatsApp Image 2025-06-05 at 17.36.52_457502cd.jpg
🐄 Predicted Cow ID: COW (11)
🎯 Confidence: 49.58%
